In [25]:
import json

import pandas as pd
import nltk
import requests

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')


client_cert = "20250129_IDU_ADGX_LeonTur.crt"
ca_cert = "onti-ca.crt"
client_key = "DECFILE"

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dash\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dash\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Dash\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [26]:
from nltk.tokenize import word_tokenize

text = "Лемматизированная форма слова листья это лист"
tokens = word_tokenize(text)
print(tokens)

['Лемматизированная', 'форма', 'слова', 'листья', 'это', 'лист']


In [9]:
df = pd.read_csv("with_names.csv")

In [11]:
df["tags"] = df["text"].apply(word_tokenize)

In [13]:
import re

def clea_from_non_alpha(words: list[str]) -> list[str]:

    cleaned_list = []
    for i in words:
        cleaned_word = re.sub(r'[^a-zA-Zа-яА-ЯёЁ0-9]', '', i)
        if cleaned_word:
            cleaned_list.append(cleaned_word)

    return cleaned_list

In [14]:
df["tags"] = df["tags"].apply(clea_from_non_alpha)

In [18]:
def construct_lemma_prompt(text: str) -> str:

    prompt = f"""
        Приведи слово "{text}" в первоначальную форму.
        В качестве ответа дай только это слово в первоначальной форме.
        Если слово не было предоставлено, то в качестве овтета дай ""
        """
    return prompt

In [19]:
def request_from_llm(prompt: str) -> str:
    """
    Function extracts prompt from llm api
    Args:
        prompt (str): prompt to execute
    Returns:
        str: request from llm
    """

    headers = {
        "Content-Type": "application/json"
    }
    data = {
        "model": "llama3.3",
        "prompt": prompt,
        "stream": False,
    }
    response = requests.post(
        url="http://a.dgx:11434/api/generate",
        headers=headers,
        data=json.dumps(data),
        cert=(client_cert, client_key),
        verify=ca_cert
    )
    if response.status_code == 200:
        return response.json()["response"]
    return ""

In [ ]:
def get_tags(text: str) -> str:

    prompt = construct_fform(text)
    tags_from_llm = request_from_llm(prompt)
    return tags_from_llm

In [ ]:
res_tags = []
for i in tokens:
    fform =